In [ ]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import expit

# 2D classification

In [ ]:
from project2_tools import get_2Ddata, to_onehot, from_onehot
from neuralnet import NeuralNet

In [ ]:
data_input, data_targets = get_2Ddata(t_crit = 2.3, onehot = True)

In [ ]:
from sklearn.model_selection import train_test_split
temp = train_test_split(data_input, data_targets, test_size = 0.33)
input_train, input_test, target_train, target_test = temp

In [ ]:
layer_sizes = [1600,100, 2]
net = NeuralNet(layer_sizes, act_func = ['sigmoid', 'softmax'], net_type = 'classifier')

In [ ]:
from result_functions import run_minibatch_sgd_one_hidden as sgd
sgd(input_train,target_train, input_test, target_test, eta_values = np.logspace(-0.5,2,2), 
    n_hidden_values = [8,10,12,14])

In [ ]:
zs, outputs = net.feed_forward_vectorized(input_test)
to_onehot( np.argmax(outputs[-1], axis = 1))

In [ ]:
net = NeuralNet(layer_sizes, act_func = ['sigmoid', 'softmax'], net_type = 'classifier')

N = 2000
x = input_train[:N]
y = target_train[:N]

%lprun -f net.backpropagate_vectorized net.backpropagate_vectorized(x, y, vector_input = True)

In [ ]:
%lprun -f net.feed_forward_vectorized net.update_batch_vectorized(x, y, eta = 0.01)

In [ ]:
import time

btimes = []
ftimes = []
N_val = np.logspace(1,4,10, dtype = int)

for N in N_val:
    net = NeuralNet(layer_sizes, act_func = ['sigmoid', 'softmax'], net_type = 'classifier')

    x = input_train[:N]
    y = target_train[:N]
    start = time.time()
    net.backpropagate_vectorized(x, y, vector_input = True)
    stop = time.time()
    btimes.append(stop-start)
    start = time.time()
    
    net.feed_forward_vectorized(x)
    stop = time.time()
    ftimes.append(stop-start)
    

In [ ]:
plt.loglog(N_val, np.array(btimes)-ftimes, label = 'Backprop - FF')
plt.loglog(N_val, ftimes, label = 'Feed Forward')
plt.legend()


In [ ]:
%lprun -f net.feed_forward_vectorized net.backpropagate_vectorized(x, y, vector_input = True)

In [ ]:
# a,b,a2,a3  = net.backpropagate_vectorized(x, y, vector_input = True)

In [ ]:

# %timeit np.einsum('ij,ik->jk', a, b, out = c1)
# %timeit np.einsum('...j,...k->...jk', a, b, out = c2)

In [ ]:
# from project2_tools import calc_sum, calc_sum2, calc_sum3, add_outer_products
# N = a.shape[0]
# %timeit sum_outer_product(a, b, c1, weight = 1/N)

In [ ]:
a = np.tensordot(x, net.weights[0], axes = [1,1]) + net.biases[0]
b = np.einsum('...ij,...j->...i',net.weights[0] , x) + net.biases[0]
c = np.matmul(net.weights[0], x.T).T + net.biases[0]
d = np.matmul(x, net.weights[0].T) + net.biases[0]

[np.all(np.isclose(a, other)) for other in [b,c,d]]

In [ ]:
print('matmul')
%timeit np.matmul(net.weights[0], x.T).T 
print('matmul, x first')
%timeit np.matmul(x, net.weights[0].T) 
print('tensordot')
%timeit np.tensordot(x, net.weights[0], axes = [1,1]) 
print('einsum')
%timeit np.einsum('...ij,...j->...i',net.weights[0] , x) 

In [ ]:
np.tensordot(x, net.weights[0], axes = [1,1]).shape, x.shape, net.weights[0].shape

In [ ]:
np.dot(x, net.weights[0].T).shape, x.shape, net.weights[0].shape

In [ ]:
%timeit np.tensordot(x, net.weights[0], axes = [1,1]) 

In [ ]:
%timeit np.dot(x, net.weights[0].T)

In [ ]:
from IPython.display import clear_output
i = 0

net = NeuralNet(layer_sizes, act_func = ['sigmoid', 'softmax'], net_type = 'classifier')

eta = 1e-3
mse = []
n_epochs = 100
n_batches = 10000

tot = n_epochs * n_batches
current = 1

import time
start = time.time()

for j in range(n_epochs):
    b = batches_vectorized(input_train, target_train, n_batches = n_batches)
    for k, (x,y) in enumerate(b):
        clear_output(wait = True)
        print('batch, epoch, MSE')
        print('{:5}/{}  {:5}/{}  {:.2f}'.format(k, n_batches, j, n_epochs, mse[-1] if len(mse) else 0))#, 'o', markersize = 9)
        now = time.time() 
        print('Time estimate: {:.0f} seconds left'.format((now - start)/current * (tot-current)))
        net.update_batch_vectorized(x, y, eta)
    
        current += 1

    mse.append(np.average(test_net(input_test, target_test, net)))
    if np.isnan(mse[-1]) or np.any([np.any(np.isnan(w)) for w in net.weights]):
        print('NAN!!! Break! Abort mission!')
        break

In [ ]:
from IPython.display import clear_output
i = 0

eta = 1e-3
mse = []
n_epochs = 100
n_batches = 10000

tot = n_epochs * n_batches
current = 1

import time
start = time.time()

for j in range(n_epochs):
    b = batches(input_train, target_train, n_batches = n_batches)
    for k, batch in enumerate(b):
        clear_output(wait = True)
        print('batch, epoch, MSE')
        print('{:5}/{}  {:5}/{}  {:.2f}'.format(k, n_batches, j, n_epochs, mse[-1] if len(mse) else 0))#, 'o', markersize = 9)
        now = time.time() 
        print('Time estimate: {:.0f} seconds left'.format((now - start)/current * (tot-current)))
        net.update_batch(batch, eta)
    
        current += 1

    mse.append(np.average(test_net(input_test, target_test, net)))
    if np.isnan(mse[-1]) or np.any([np.any(np.isnan(w)) for w in net.weights]):
        print('NAN!!! Break! Abort mission!')
        break

In [ ]:
def f(x,y,z):
    return x+y+z

a = [1,2,3]

f(*a)

$\in$

In [ ]:
from project2_tools import softmax

%timeit net.feed_forward(input_train[0])


In [ ]:
net.backpropagate(input_train[0], target_train[0])

In [ ]:
plt.plot(mse)

In [ ]:
from nielsen_network import Network as nNetwork

net2 = nNetwork(layer_sizes)
net2.backprop(np.atleast_2d(x),np.atleast_2d(y))
# x,y = batch[0]

In [ ]:
np.atleast_2d(x).shape

In [ ]:
y

find last delta:
$$ \delta_j^L = f'(z_j^L) \frac{\partial \mathcal{C}}{\partial a_j^L} $$

propagate deltas for each layer
$$ \delta_j^l = \sum_k \delta_k^{l+1} w_{kj}^{l+1} f'(z_j^l)$$

update weights and bias
$$w_{jk}^l \leftarrow w_{jk}^l - \eta \delta_j^l a_k^{l-1} $$
$$b_{j}^l \leftarrow b_{j}^l - \eta \delta_j^l  $$



we have

$$ \frac{\partial \mathcal{C}}{\partial a_j^L} = a_j^L - t_j, $$
and
$$ f'(z_j^l) = f(z_j^l)  (1 - f(z_j^l) $$
for sigmoid

In [ ]:
for i, batch in enumerate(batches(input_train, target_train, n_batches = 100)):
    break
    

In [ ]:
x,y = batch[0]
x,y

In [ ]:
net = NeuralNet(layer_sizes, act_func = ['sigmoid','identity'])

for k in range(20):
    for i, batch in enumerate(batches(input_train, target_train, n_batches = 100)):
        net.update_batch(batch, eta = 1e-2)

    plt.plot(k, np.average(test_net(input_test, target_test, net)), 'o', markersize = 10)

In [ ]:
fig,[ax1,ax2] = plt.subplots(1,2, figsize = [8,4])
i = 43
test_out = np.array([net.feed_forward(inp)[1][-1] for inp in input_test]).squeeze()
ax1.scatter(test_out, target_test)
ax1.set_xlabel('predicted')
ax1.set_ylabel('target')

ax1.plot([-12,12],[-12,12])
ax1.axis('equal')


#ax2.hist(test_out,  alpha = 0.5)
#ax2.hist(target_test,  alpha = 0.5);